<a href="https://colab.research.google.com/github/manuvillalba-uclm/Natural_Lenguaje_Processing-ELM/blob/main/Machine_Learning_NLP_PreProcesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TO DO


Los emojis los traduce bien pero no como queremos, nos da igual que nos diga face_with_tears_of_joy, el objetivo es que tears y joy se integren con el texto, eso hay que revisarlo.

No sé hasta que punto nos interesa mantener los hastag




In [ ]:
# dataframes library
!pip install pyspellchecker 
!pip install contractions
!pip install nltk --upgrade
!pip install emoji

import pandas as pd
import re
import nltk
nltk.download("popular")
nltk.download('vader_lexicon') #sentiment analysis
nltk.download('twython') #twitter not necessary

from nltk.tokenize.casual import TweetTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

#Correcciones y emoji
import emoji
from textblob import TextBlob
from spellchecker import SpellChecker
import contractions
from multiprocessing import Pool

#Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

#Feature selection 
from sklearn.feature_selection import SelectKBest, chi2

#Classification
from sklearn.model_selection import train_test_split

from sklearn import neighbors
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels



In [ ]:
english_stops = set(stopwords.words('english'))
spell = SpellChecker()
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()


In [ ]:
raw_dataset_tweets = pd.read_csv('https://raw.githubusercontent.com/manuvillalba-uclm/Natural_Lenguaje_Processing-ELM/main/labeled_data.csv?token=ANLXPWL27WHT4VS5U4LHMVK72D27M', index_col=[0])

# Preprocesing:

Mandatory preprocessing steps
 - Remove unseful data: ! "_ $% & / ( ) = _ ˆ* ¡@
 - Remove all capital letters
 - Correct wrong words: https://norvig.com/spell-correct.html
 - Lemmatize all terms

Optional preprocessing steps:
 - Remove contractions: don’t -> do not
 - Remove repeated words: great great show
 - Replace emoticons, for example,🙂 with “smile”, 😉with “ok”


We just need index and tweet content for this first part, so we drop Crowdflowers users information from the dataset.

In [ ]:
dataset_tweets_preprocesing = raw_dataset_tweets.copy()
dataset_tweets_preprocesing.drop("count", axis = 1, inplace = True)
dataset_tweets_preprocesing.drop("hate_speech", axis = 1, inplace = True)
dataset_tweets_preprocesing.drop("offensive_language", axis = 1, inplace = True)
dataset_tweets_preprocesing.drop("neither", axis = 1, inplace = True)
dataset_tweets_preprocesing.drop("class", axis = 1, inplace = True)

In [ ]:
dataset_tweets_preprocesing

Seleccionamos sólo unos pocos para trabajar al principio más fácilmente

In [ ]:
#@title Texto de título predeterminado
num_tweets = 24783 #@param {type:"slider", min:20, max:24783, step:10}

examples_preprocesing = dataset_tweets_preprocesing.iloc[0:num_tweets,0]
examples_preprocesing.reset_index(inplace = True, drop= True)
print(examples_preprocesing)

lista_tweets = []



### Definimos las funciones que procesarán los tweets


In [ ]:
def stopwords(text):
    return " ".join([word for word in str(text).split() if word not in english_stops])

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def demojizar(arr):
    text_arr = []
    for token in arr:
      text_arr.append(emoji.demojize(token, delimiters=("", "")))
    return text_arr

def textConditions(token):
  if re.match("[`!\"_$%&/()=_ˆ*¡,@:.]+",token): #Signos de puntuacion
    return False
  if re.match("[-><~]+",token): #Signos de puntuacion
    return False
  if re.match("[^\u0000-\u007F]+",token): #Unicodes
    return False
  return True

def processArray(arr):
  text_arr = []
  for token in arr:
    if textConditions(token):
      token = stemmer.stem(token)
      token = lemmatizer.lemmatize(token) # < ---- LEMMA
      text_arr.append(token)
  # try: #Aqui iria el tagger de palabras, pero solo me lo hace para letras (ERROR)
  #   arr_tags = nltk.pos_tag(text_arr)
  # except:
  #   arr_tags = text_arr
  return text_arr

In [ ]:
%%time

ds_alt_twitter = examples_preprocesing.copy()
ds_alt_twitter = ds_alt_twitter.str.lower() #A minusc
ds_alt_twitter = ds_alt_twitter.apply(contractions.fix) #Arreglar contracciones
ds_alt_twitter = ds_alt_twitter.apply(contractions.fix) #Plural a singular

ds_alt_twitter = ds_alt_twitter.str.replace('(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z0-9_]+[A-Za-z0-9-_]+)','') #Remover usuario
ds_alt_twitter = ds_alt_twitter.str.replace('(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z0-9_]+[A-Za-z0-9-_]+)','') #Remover usuario
ds_alt_twitter = ds_alt_twitter.str.replace('rt','') #Remover rt

ds_alt_twitter = ds_alt_twitter.apply(stopwords) #Quitar Stopwords
ds_alt_twitter = ds_alt_twitter.apply(remove_urls) #Quitar url

ds_alt_twitter = ds_alt_twitter.apply(lambda x: str(TextBlob(x).correct()))
ds_alt_twitter = ds_alt_twitter.apply(TweetTokenizer().tokenize) #Tokenizar
ds_alt_twitter = ds_alt_twitter.apply(demojizar) #Traducir emoji a texto
ds_alt_twitter = ds_alt_twitter.apply(processArray) #Traducir emoji a texto



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

ds_alt_twitter.to_csv(r'/content/gdrive/My Drive/preprocesado.csv')